# Web Scraping con Python
Rafa Caballero

Este script muestra algunas posibilidades de la biblioteca *Selenium* que nos permite obtener, automáticamente, datos de páginas web dinámicas, es decir que necesitan pulsar botones, etc, etc.

Selenium está pensado realmente para automatizar pruebas de entornos web, pero a nosotros nos servirá para nuestro propósito de hacer web scraping.

En particular, lo que vamos a hacer es arrancar Google Chrome y manejarlo automáticamente desde Python.

Para ello necesitamos 3 cosas:

1.- Tener google Chrome

2.- Instalar la biblioteca selenium

3.- Tener un fichero controlador, un driver, de nombre *chromedriver.exe*. En nuestro caso supondremos que se encuentra en la carpeta c:/hlocal/sgdi.

Podemos obtener el fichero Chromedriver, desde http://chromedriver.storage.googleapis.com/index.html

Debemos descargar un driver suficientemente moderno, según nuestra versión de Chrome.

Para encontrar la versión de Chrome, desde dentro del navegador :
-	Abrir el menú de arriba a la derecha (icono 3 puntitos verticales, o una “i”)
-	Selecciones “Ayuda” y a continuación “Información de Google Chrome”
-	Allí debemos ver algo como “Versión 76.0.3809.132 (Build oficial) (64 bits)”


Empezamos por asegurarnos de que Selenium está instalado, e instalándolo en caso contrario

In [1]:
import sys
try:
    import selenium
    print("selenium está en el sistema!")
except ImportError as e:
    !{sys.executable} -m pip install --upgrade --user selenium

selenium está en el sistema!


Iniciamos una sesión de Chrome, recordar cambiar la variable al lugar donde está el fichero Chromedriver

In [2]:
import os
import time
from selenium import webdriver  # si da error, desde anaconda prompt sobre python 3.x hacer pip install --user  selenium
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

chromedriver = "/usr/local/share/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')

#driver = webdriver.Chrome(executable_path=chromedriver,options=chrome_options)
driver = webdriver.Chrome( service=Service(ChromeDriverManager().install()), options=chrome_options)


[WDM] - Downloading: 100%|███████████| 6.98M/6.98M [00:00<00:00, 17.6MB/s]


##### Ejercicio 1

Cargar la página http://mustcalculate.com/electronics/unitconverter.php

In [3]:
s = 'http://mustcalculate.com/electronics/unitconverter.php'
driver.get(s)



##### Ejercicio 2
Cargar los valores a convertir (convertir.csv) como un Dataframe pandas

In [4]:
import pandas as pd
valores = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/convertir.csv"
df = pd.read_csv(valores,encoding= 'unicode_escape')
df

,valor,origen,destino
0,10,Binary,Octal
1,100,Fahrenheit,Celsius
2,1000,Hertz,Seconds


#### Ejercicio 3
Para cada fila:

* Poner en Value el `valor`
* En la lista desplegable elegir el Origen
* Dar al botón `Calculate`
* Buscar el resultado en la tabla Results y añadirlo a una lista

Al final añadir una columna 'resultado' al dataframe y volver a grabarlo con formato csv

Sugerencia: hacerlo al principio solo para la primera fila, y cuando funcione iterar en un bucle

Para localizar los elementos mirar esta [ayuda](https://selenium-python.readthedocs.io/locating-elements.html)

Para seleccionar un elemento de un desplegable utilizar Select. [Aquí](https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python) hay un ejemplo (buscar la respuesta que utiliza Select)

In [13]:
from selenium.webdriver.support.ui import Select

# para guardar los resultados
resultados=[]
for index, r in df.iterrows():
    # meter los valores en la caja de texto
    caja = driver.find_element(By.NAME, "in")
    caja.clear()
    caja.send_keys(str(r.valor))

    # lista desplegable
    select = Select(driver.find_element(By.NAME,'ino'))
    select.select_by_visible_text(r.origen)

    # pulsar botón
    driver.find_element(By.ID,'calcbutton').click()

    # buscar en la tabla
    div = driver.find_element(By.ID,'answer')
    trs = div.find_elements(By.XPATH,"table/tbody/tr")
    
    found = False
    i = 0
    while not found and i<len(trs): 
        tr = trs[i]
        # ojo con la cabecera, que tiene otro formato
        if len(tr.find_elements(By.XPATH, "td"))>1:
            nombre = tr.find_element(By.XPATH, "td[1]").text
            valor = tr.find_element(By.XPATH, "td[2]").text
            if nombre==r.destino:
                resultados.append(valor)
                found=True
        i+=1
df["resultados"] = resultados
print(df)
df.to_csv(r'resultados.csv', index=False)

   valor      origen  destino      resultados
0     10      Binary    Octal               2
1    100  Fahrenheit  Celsius  37.77777777778
2   1000       Hertz  Seconds           0.001


In [14]:
driver.close()